In [1]:
from utils import UNet, SegmentationDataset
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn

dataset = SegmentationDataset("SA1B_Meta_AI_Segmentation_Dataset/")
dataloader = DataLoader(dataset, batch_size=50, shuffle=True)
model = UNet(in_channels=3, out_channels=1)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [2]:
def train_unet(model, dataset, epochs=10, lr=1e-4):
    dataloader = DataLoader(dataset, batch_size=16, shuffle=True)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCELoss()  # For binary segmentation

    model.train()
    for epoch in range(epochs):
        epoch_loss = 0.0
        for imgs, masks in dataloader:
            imgs, masks = imgs.cuda(), masks.cuda()  # Move to GPU if available

            # Forward pass
            preds = model(imgs)
            loss = criterion(preds, masks.unsqueeze(1))  # Add channel dim to masks
            epoch_loss += loss.item()

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / len(dataloader)}")

    return model

In [ ]:
model.train()
for epoch in range(2):
	epoch_loss = 0
	for idx, (image, mask) in enumerate(dataloader):
		outputs = model(image)
		loss = criterion(outputs, mask)
		optimizer.zero_grad()
		loss.backward()
		optimizer.step()
		epoch_loss += loss.item()
		# outputs.weights
		print(f'epoch: {epoch+1} | loss: {epoch_loss}')


In [ ]:
print(dataset)

In [1]:
import cherrypy


class TryOnApp(object):
    def __init__(self, reference_image):
        self.reference_image = reference_image
        
    @cherrypy.expose
    def display_reference(self):
        cherrypy.response.headers['Content-Type'] = "image/jpeg"
        with open(self.reference_image, "rb") as f:
                return f.read()

    @cherrypy.expose
    def upload_image(self, image=None):
        if image is not None:
            # 'image' will be a file-like object
            uploaded_data = image.file.read()
            uploaded_filename = image.filename
            return f"Uploaded file: {uploaded_filename}, size: {len(uploaded_data)} bytes"
        else:
            return '''
            <html>
                <body>
                    Hello
                </body>
            </html>
            '''

    @cherrypy.expose
    def index(self):
        return '''
        <html>
            <head>
                <link href="front_end/styles/styles.css" rel="stylesheet">
            </head>
            <body>
                <h1>Image Display</h1>
                <img src="/display_reference" alt="My Image">
                <form action="upload" method="post" action="upload_image" enctype="multipart/form-data">
                        <input type="file" name="image" />
                        <button type="submit">Upload</button>
                </form>
            </body>
        </html>
        '''
        

In [2]:
import os

ref_img = os.path.relpath('front_end/Rebecca-Ingram-Adeline-Fit-and-Flare-Wedding-Dress-25RK278A01-PROMO1-IV.jpg')
usr_img = os.path.relpath('flexing_01.png')

In [3]:
if __name__ == '__main__':
    cherrypy.quickstart(TryOnApp(reference_image=ref_img))

[25/Jan/2025:16:51:41] ENGINE Listening for SIGTERM.
[25/Jan/2025:16:51:41] ENGINE Listening for SIGHUP.
[25/Jan/2025:16:51:41] ENGINE Listening for SIGUSR1.
[25/Jan/2025:16:51:41] ENGINE Bus STARTING
CherryPy Checker:
The Application mounted at '' has an empty config.

[25/Jan/2025:16:51:41] ENGINE Started monitor thread 'Autoreloader'.
[25/Jan/2025:16:51:41] ENGINE Serving on http://127.0.0.1:8080
[25/Jan/2025:16:51:41] ENGINE Bus STARTED


127.0.0.1 - - [25/Jan/2025:16:51:45] "GET / HTTP/1.1" 200 408 "" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36"
127.0.0.1 - - [25/Jan/2025:16:51:45] "GET /display_reference HTTP/1.1" 200 138708 "http://127.0.0.1:8080/" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36"
127.0.0.1 - - [25/Jan/2025:16:51:45] "GET /favicon.ico HTTP/1.1" 200 1406 "http://127.0.0.1:8080/" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36"


[25/Jan/2025:16:53:27] ENGINE Keyboard Interrupt: shutting down bus
[25/Jan/2025:16:53:27] ENGINE Bus STOPPING
[25/Jan/2025:16:53:27] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('127.0.0.1', 8080)) shut down
[25/Jan/2025:16:53:27] ENGINE Stopped thread 'Autoreloader'.
[25/Jan/2025:16:53:27] ENGINE Bus STOPPED
[25/Jan/2025:16:53:27] ENGINE Bus EXITING
[25/Jan/2025:16:53:27] ENGINE Bus EXITED
[25/Jan/2025:16:53:27] ENGINE Waiting for child threads to terminate...


In [ ]:
import cherrypy
from PIL import Image, ImageDraw
import io

class MyApp:
    @cherrypy.expose
    def index(self):
        return '''
        <html>
            <body>
                <h1>Dynamic Image</h1>
                <img src="/dynamic_image" alt="Dynamic Image">
            </body>
        </html>
        '''

    @cherrypy.expose
    def dynamic_image(self):
        # Create an image dynamically
        image = Image.new("RGB", (200, 100), color=(73, 109, 137))
        draw = ImageDraw.Draw(image)
        draw.text((10, 10), "Hello, CherryPy!", fill=(255, 255, 0))

        # Save the image to an in-memory bytes buffer
        buffer = io.BytesIO()
        image.save(buffer, format="PNG")
        buffer.seek(0)

        # Serve the image
        cherrypy.response.headers['Content-Type'] = "image/png"
        return buffer.read()

# if __name__ == '__main__':
#     cherrypy.quickstart(MyApp())